## Siamese Network
- 特殊的模型架構，常用於相似度的比較，分為模型參數分享與不分享類型，各有各的好處。
    - 句子相似度。
    - 人臉辨識。
    - 以圖搜圖。
- [參考](https://zhuanlan.zhihu.com/p/35040994)

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# 距離函數，可根據實際狀況任意不同function
l1_norm = lambda x: 1 - keras.backend.abs(x[0]-x[1])  # element-wise abs

def get_siamese_model(**kwargs):
    input1 = keras.Input(shape=kwargs['shape'])
    input2 = keras.Input(shape=kwargs['shape'])
    embedding1 = layers.Embedding(input_dim=kwargs['input_dim'], output_dim=kwargs['output_dim'])(input1)
    embedding2 = layers.Embedding(input_dim=kwargs['input_dim'], output_dim=kwargs['output_dim'])(input2)
    
    lstm = layers.Bidirectional(layers.LSTM(units=256, return_sequences=False))
    
    embedding1 = lstm(embedding1)
    embedding2 = lstm(embedding2)
    
    merged = layers.Lambda(function=l1_norm, output_shape=lambda x: x[0], name='L1_distance')([embedding1, embedding2])
    
    output = layers.Dense(1, activation='sigmoid', name='classifier_layer')(merged)
    
    model = keras.Model([input1, input2], output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    print(model.summary())
    
    return model
    
config = {
    'shape': (128,),
    'input_dim':  30000,
    'output_dim': 128
}

siamese_model = get_siamese_model(**config)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 128, 128)     3840000     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 128, 128)     3840000     input_4[0][0]                    
______________________________________________________________________________________________